In [1]:
%graph_notebook_host growsmart-neptune.cluster-custom-cgogeml0cuty.eu-west-3.neptune.amazonaws.com

set host to growsmart-neptune.cluster-custom-cgogeml0cuty.eu-west-3.neptune.amazonaws.com


In [18]:
%%oc
//Execute this cell (uncomment query) to delete the full graph to load again we have to trigger the bulk loader
//
// PLEASE IF YOU USE THIS CELL TO DROP THE GRAPH LET THE REST OF US KNOW
//
//MATCH (n)
//DETACH DELETE n

In [36]:
%%oc
MATCH(n:Sensor)-[c]->(s:Event)-[c2]->(d:SensorData)
RETURN n, c, s, c2, d
LIMIT 200

In [ ]:
import python-cypher

connector = Connector(host, port)
query = """
MATCH(n:Sensor)-[c]->(s:Event)-[c2]->(d:SensorData)
RETURN n, c, s, c2, d
LIMIT 200
"""



In [1]:
%db_reset --generate-token # This command will reset the db, run at your own risk

{'status': '200 OK',
 'payload': {'token': '80c44ba8-ae32-ce6b-3b20-638f527ed25d'}}

In [ ]:
%db_reset --token 80c44ba8-ae32-ce6b-3b20-638f527ed25d # This command will reset the db, run at your own risk

{'status': '200 OK'}

In [56]:
%%oc
MATCH (p:Plant)
SET p.status = 1.0 //Healthy
RETURN p

In [57]:
%%oc
MATCH (p:Plant)<-[m:MEASURES]-(s:Sensor)-[r:REGISTERS]->(e:Event)-[c:CONTAINS]->(sd:iot_soil_ph)
OPTIONAL MATCH (e1:Event)-[c1:CONTAINS]->(sd1:iot_soil_humidity)
WITH p, m, s, r, e, e1, c, c1, toFloat(sd.value) AS ph, toFloat(sd1.value) AS hum
WHERE e=e1 AND (ph < 5.0 OR ph > 7.0) AND p IS NOT NULL AND (hum < 60.0)
SET p.status = 0.0 //Unhealthy
RETURN DISTINCT ID(p)

In [46]:
%%oc
MATCH (n:iot_soil_humidity)
RETURN n.value ORDER BY n.value DESC

In [52]:
%%oc
MATCH (p:Plant)
WITH p, rand() AS r
ORDER BY r
LIMIT 25
SET p.status = 0.0
RETURN p

In [ ]:
%%oc
MATCH (p:Plant)<-[m:MEASURES]-(s:Sensor)-[r:REGISTERS]->(e:Event)-[c:CONTAINS]->(sd)
WITH p, m, s, r, e, c, sd
WHERE sd['value'] IS NOT NULL AND labels(sd)[0] <> 'iot_light' AND labels(sd)[0] <> 'iot_motion'
RETURN sd['value'] AS value, labels(sd)[0] AS value_label